# Генерация признаков и формирование обучающих выборок (TF-IDF vs BERT)

**Цель ноутбука.**  
Сформировать обучающие/тестовые выборки для двух вариантов контент-признаков постов:
1) **TF-IDF + PCA** (контрольная ветка),
2) **BERT-эмбеддинги + PCA** (тестовая ветка)

**Входные данные:**
- `user.csv`, `post.csv`, `feed_sampled.csv`,
- `embeddings_post.csv` (BERT-эмбеддинги постов, получены в отдельном ноутбуке).

**Выходные данные:**
- `X_train_tfidf.csv`, `y_train_tfidf.csv`, `X_test_tfidf.csv`, `y_test_tfidf.csv`
- `X_train_bert.csv`,   `y_train_bert.csv`,   `X_test_bert.csv`,  `y_test_bert.csv`
- таблицы в БД (Postgres):  
  - `tikhonovrs96_features_user_lesson_22`  
  - `tikhonovrs96_features_post_tfidf_lesson_22`  
  - `tikhonovrs96_features_post_bert_lesson_22`  
  - `tikhonovrs96_features_mte_lesson_22`

**Ключевые решения:**
- Валидация — **temporal split**: train \< `2021-12-16`, test ≥ `2021-12-16`.  
- MTE делаем через KFold на трейне, применяем маппинг к тесту.  
- `exp_group` здесь закодирован для обучения; **в прод A/B** он не используется.



In [ ]:
# Базовые импорты для обработки данных и визуализаци
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 80, 'display.max_columns', 50)

In [ ]:
# Глобальные стили matplotlib
import matplotlib as mlp

mlp.rcParams['lines.linewidth'] = 5

mlp.rcParams['xtick.major.size'] = 20
mlp.rcParams['xtick.major.width'] = 5
mlp.rcParams['xtick.labelsize'] = 20
mlp.rcParams['xtick.color'] = '#FF5533'

mlp.rcParams['ytick.major.size'] = 20
mlp.rcParams['ytick.major.width'] = 5
mlp.rcParams['ytick.labelsize'] = 20
mlp.rcParams['ytick.color'] = '#FF5533'

mlp.rcParams['axes.labelsize'] = 20
mlp.rcParams['axes.titlesize'] = 20
mlp.rcParams['axes.titlecolor'] = '#00B050'
mlp.rcParams['axes.labelcolor'] = '#00B050'

In [ ]:
# Загружаем подготовленные данные
user = pd.read_csv('user.csv')
post = pd.read_csv('post.csv')
filtered_sample = pd.read_csv('feed_sampled.csv', parse_dates=['timestamp'])

In [ ]:
# Осмотрим user перед кодированием категориальных полей
user.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [ ]:
# Функция для разбивки возраста по категориям
def cat_ages(age):
    if age < 18:
        return 'до 18'
    if age < 25:
        return "18-25"
    if age < 35:
        return "25-35"
    if age < 45:
        return "35-45"
    if age < 55:
        return "45-55"
    if age < 65:
        return "55-65"
    if age < 75:
        return "65-75"
    else:
        return "75 и выше"

In [ ]:
# Перекодируем возраст в интервальные категории
user['age'] = user['age'].apply(cat_ages)

In [ ]:
# Посмотрим на source —> пригодится для бинаризации
user['source'].value_counts()

ads        101685
organic     61520
Name: source, dtype: int64

In [ ]:
# Бинарные индикаторы по os/source; затем удаляем исходные текстовые поля
user['is_android'] = user['os'].apply(lambda x: 1 if x == 'Android'
                                      else 0)
user['is_ads'] = user['source'].apply(lambda x: 1 if x == 'ads' 
                                      else 0)
user = user.drop(['os', 'source'], axis=1)

In [ ]:
# OHE для exp_group; ВАЖНО: в прод A/B не использовать, там разбиение по md5+соль
ohe_exp_group = pd.get_dummies(user['exp_group'], prefix='exp_group', drop_first=True)
user = pd.concat((user.drop('exp_group', axis=1), ohe_exp_group), axis=1)

In [ ]:
# Проверяем результат кодирования фичей пользователя
user.head()

,user_id,gender,age,country,city,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4
0,200,1,25-35,Russia,Degtyarsk,1,1,0,0,1,0
1,201,0,35-45,Russia,Abakan,1,1,0,0,0,0
2,202,1,до 18,Russia,Smolensk,1,1,0,0,0,1
3,203,0,18-25,Russia,Moscow,0,1,1,0,0,0
4,204,0,35-45,Russia,Anzhero-Sudzhensk,1,1,0,0,1,0


In [11]:
# 1. TF-IDF + PCA вариант (контрольная модель)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

vectorizer = TfidfVectorizer()
X_text = vectorizer.fit_transform(post['text'])
scaler = StandardScaler()
X_text_scaled = scaler.fit_transform(X_text.toarray())
reduced = PCA(n_components=20)
X_reduced = reduced.fit_transform(X_text_scaled)
df_reduced = pd.DataFrame(X_reduced, columns=[f'pca_{i+1}' for i in range(X_reduced.shape[1])])
post_tfidf = pd.concat((post, df_reduced), axis=1)

# 2. BERT-вариант (тестовая модель)
embeddings = pd.read_csv("embeddings_post.csv")
post_bert = pd.merge(post, embeddings, on="post_id")

In [ ]:
# OHE темы поста для обеих веток (drop_first для избежания коллинеарности)
ohe_topic = pd.get_dummies(post_tfidf['topic'], prefix='topic', drop_first=True)
post_tfidf = pd.concat((post_tfidf, ohe_topic), axis=1)

ohe_topic = pd.get_dummies(post_bert['topic'], prefix='topic', drop_first=True)
post_bert = pd.concat((post_bert, ohe_topic), axis=1)

In [ ]:
# Проверим структуру таблицы взаимодействий (view с таргетом)
filtered_sample.head()

,user_id,post_id,timestamp,target
0,51533,1784,2021-10-21 11:16:14,1
1,51533,1344,2021-10-21 11:18:18,0
2,51533,1911,2021-10-21 11:19:03,0
3,51533,1622,2021-10-21 11:20:33,0
4,51533,1211,2021-10-21 11:21:33,0


## 1 вариант (контроль): TF-IDF + календарные фичи + MTE

In [ ]:
# Джойним user к логам по user_id (left — сохраняем все события)
df_tfidf = pd.merge(
    filtered_sample,
    user,
    on='user_id',
    how='left'
)

In [ ]:
# Джойним постовые фичи (TF-IDF PCA + OHE темы) по post_id
df_tfidf = pd.merge(
    df_tfidf,
    post_tfidf.drop(['text', 'topic'], axis=1),
    on='post_id',
    how='left'
)

In [ ]:
# Быстрая проверка результата объединения
df_tfidf.head()

,user_id,post_id,timestamp,target,gender,age,country,city,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,51533,1784,2021-10-21 11:16:14,1,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.181055,-0.288288,-0.422105,-0.361728,-0.823934,-1.163084,-0.457821,-0.296544,0.677534,-0.033938,0.752776,0.028988,-3.478632,5.554537,-1.735239,-0.760399,1.942214,0.490945,-2.612931,1.566407,0,0,0,0,1,0
1,51533,1344,2021-10-21 11:18:18,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.244778,1.171943,-0.487945,-1.144365,-1.489284,-2.875986,0.073866,0.703234,5.743786,2.525433,-2.229066,-2.814638,-13.018264,-6.145329,4.917475,0.076079,-2.851801,-1.665349,0.203243,-0.832864,0,0,0,1,0,0
2,51533,1911,2021-10-21 11:19:03,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.319494,-0.416234,-0.394980,-0.443572,-0.819001,-1.064639,-0.460328,-0.144719,0.380468,0.300065,-0.233650,-0.392548,-2.261238,-0.010170,-1.342536,0.695462,0.764987,0.170401,-1.089159,1.229232,0,0,0,0,1,0
3,51533,1622,2021-10-21 11:20:33,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.325991,-0.139774,-0.294887,-0.435077,-0.873326,-0.992215,-0.506083,-0.076823,0.718392,0.247727,0.390431,-0.215585,-2.890603,3.255657,-1.665595,0.161599,0.990986,0.202333,-0.833970,0.751094,0,0,0,0,1,0
4,51533,1211,2021-10-21 11:21:33,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.315246,0.320371,-0.555416,-0.796566,-1.013993,-1.516432,0.257158,0.613133,3.411011,0.957152,-0.587802,-1.599982,-7.393958,-3.175255,2.470751,0.243670,-1.128147,-0.139595,-0.258421,-0.110980,0,0,0,1,0,0


In [ ]:
# Календарные признаки из timestamp (для сезонности/паттернов поведения)
df_tfidf['day'] = df_tfidf['timestamp'].dt.day
df_tfidf['month'] = df_tfidf['timestamp'].dt.month
df_tfidf['dayofweek'] = df_tfidf['timestamp'].dt.dayofweek
df_tfidf.head()

,user_id,post_id,timestamp,target,gender,age,country,city,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,day,month,dayofweek
0,51533,1784,2021-10-21 11:16:14,1,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.181055,-0.288288,-0.422105,-0.361728,-0.823934,-1.163084,-0.457821,-0.296544,0.677534,-0.033938,0.752776,0.028988,-3.478632,5.554537,-1.735239,-0.760399,1.942214,0.490945,-2.612931,1.566407,0,0,0,0,1,0,21,10,3
1,51533,1344,2021-10-21 11:18:18,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.244778,1.171943,-0.487945,-1.144365,-1.489284,-2.875986,0.073866,0.703234,5.743786,2.525433,-2.229066,-2.814638,-13.018264,-6.145329,4.917475,0.076079,-2.851801,-1.665349,0.203243,-0.832864,0,0,0,1,0,0,21,10,3
2,51533,1911,2021-10-21 11:19:03,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.319494,-0.416234,-0.394980,-0.443572,-0.819001,-1.064639,-0.460328,-0.144719,0.380468,0.300065,-0.233650,-0.392548,-2.261238,-0.010170,-1.342536,0.695462,0.764987,0.170401,-1.089159,1.229232,0,0,0,0,1,0,21,10,3
3,51533,1622,2021-10-21 11:20:33,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.325991,-0.139774,-0.294887,-0.435077,-0.873326,-0.992215,-0.506083,-0.076823,0.718392,0.247727,0.390431,-0.215585,-2.890603,3.255657,-1.665595,0.161599,0.990986,0.202333,-0.833970,0.751094,0,0,0,0,1,0,21,10,3
4,51533,1211,2021-10-21 11:21:33,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.315246,0.320371,-0.555416,-0.796566,-1.013993,-1.516432,0.257158,0.613133,3.411011,0.957152,-0.587802,-1.599982,-7.393958,-3.175255,2.470751,0.243670,-1.128147,-0.139595,-0.258421,-0.110980,0,0,0,1,0,0,21,10,3


In [ ]:
# Оценка числа дублей с учётом таргета — предварительная диагностика
df_tfidf.drop(['user_id', 'post_id', 'timestamp'], axis=1).duplicated().sum()

5517

In [ ]:
# Оценка числа дублей без таргета — ищем конфликтные пары (одинаковые фичи, разные target)
df_tfidf.drop(['user_id', 'post_id', 'timestamp', 'target'], axis=1).duplicated().sum()

7372

In [ ]:
# Удаляем точные дубликаты
df_tfidf = df_tfidf.drop_duplicates(subset=df_tfidf.columns.difference(['user_id', 'post_id']))

In [ ]:
# Находим конфликтные дубли (одинаковые фичи, разные таргеты) и убираем их полностью
duplicated_indices = df_tfidf.duplicated(subset=df_tfidf.columns.difference(['user_id', 'post_id', 'timestamp', 'target']), keep=False)
df_tfidf = df_tfidf[~duplicated_indices]

In [ ]:
# Перепроверяем дубли с учётом таргета — должно быть 0
df_tfidf.drop(['user_id', 'post_id', 'timestamp'], axis=1).duplicated().sum()

0

In [ ]:
# Перепроверяем дубли без таргета — также 0
df_tfidf.drop(['user_id', 'post_id', 'timestamp', 'target'], axis=1).duplicated().sum()

0

In [ ]:
# Диапазон дат в сэмпле — для настройки temporal split
min(df_tfidf['timestamp']), max(df_tfidf['timestamp'])

(Timestamp('2021-10-01 06:01:40'), Timestamp('2021-12-29 23:43:15'))

In [ ]:
# Разбивка по времени: train < 2021-12-16, test >= 2021-12-16 (имитируем будущее)
df_train_tfidf = df_tfidf[df_tfidf['timestamp'] < '2021-12-16'].drop('timestamp', axis=1)
df_test_tfidf = df_tfidf[df_tfidf['timestamp'] >= '2021-12-16'].drop('timestamp', axis=1)

In [ ]:
# Проверка структуры train-части до кодирования MTE
df_train_tfidf.head()

,user_id,post_id,target,gender,age,country,city,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,day,month,dayofweek
0,51533,1784,1,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.181055,-0.288288,-0.422105,-0.361728,-0.823934,-1.163084,-0.457821,-0.296544,0.677534,-0.033938,0.752776,0.028988,-3.478632,5.554537,-1.735239,-0.760399,1.942214,0.490945,-2.612931,1.566407,0,0,0,0,1,0,21,10,3
1,51533,1344,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.244778,1.171943,-0.487945,-1.144365,-1.489284,-2.875986,0.073866,0.703234,5.743786,2.525433,-2.229066,-2.814638,-13.018264,-6.145329,4.917475,0.076079,-2.851801,-1.665349,0.203243,-0.832864,0,0,0,1,0,0,21,10,3
2,51533,1911,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.319494,-0.416234,-0.394980,-0.443572,-0.819001,-1.064639,-0.460328,-0.144719,0.380468,0.300065,-0.233650,-0.392548,-2.261238,-0.010170,-1.342536,0.695462,0.764987,0.170401,-1.089159,1.229232,0,0,0,0,1,0,21,10,3
3,51533,1622,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.325991,-0.139774,-0.294887,-0.435077,-0.873326,-0.992215,-0.506083,-0.076823,0.718392,0.247727,0.390431,-0.215585,-2.890603,3.255657,-1.665595,0.161599,0.990986,0.202333,-0.833970,0.751094,0,0,0,0,1,0,21,10,3
4,51533,1211,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.315246,0.320371,-0.555416,-0.796566,-1.013993,-1.516432,0.257158,0.613133,3.411011,0.957152,-0.587802,-1.599982,-7.393958,-3.175255,2.470751,0.243670,-1.128147,-0.139595,-0.258421,-0.110980,0,0,0,1,0,0,21,10,3


In [ ]:
# Функция MTE с KFold (кодируем на трейне, применяем к тесту)
from sklearn.model_selection import KFold

def mean_target_encoding(train, test, col_name, target_name="target", n_folds=5):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    mean_encoded_col = np.zeros(train.shape[0])

    for train_idx, val_idx in kf.split(train):
        train_fold, val_fold = train.iloc[train_idx], train.iloc[val_idx]

        # Вычислим среднеее по таргету для текущих фолдов
        means = train_fold.groupby(col_name)[target_name].mean()

        # Присвоим кодировки для валидационной выборки
        mean_encoded_col[val_idx] = val_fold[col_name].map(means)
    
    # Добавим закодированный столбец в тренировочную выборку
    train[f"{col_name}_mte"] = mean_encoded_col

    # Для теста применим кодировку, полученную на трейне
    global_means =train.groupby(col_name)[target_name].mean()
    test[f"{col_name}_mte"] = test[col_name].map(global_means)

    # Заполним наны средним по всем данным
    overall_mean = train[target_name].mean()
    test[f"{col_name}_mte"].fillna(overall_mean, inplace=True)

    # Дропаем оригинальные колонки
    train.drop([col_name], axis=1, inplace=True)
    test.drop([col_name], axis=1, inplace=True)

    return train, test, global_means.to_dict(), overall_mean

In [ ]:
# Список категориальных полей для MTE + сбор маппингов в отдельный датафрейм
columns_to_encode = ['country', 'city', 'age']
mte_records = []

In [ ]:
# Применяем MTE последовательно и сохраняем маппинги (включая DEFAULT)
for col in columns_to_encode:
    df_train_tfidf, df_test_tfidf, mapping_tfidf, default_tfidf = mean_target_encoding(df_train_tfidf, df_test_tfidf, col)

    # Добавим значения в список записей
    for value, enc in mapping_tfidf.items():
        mte_records.append({
            'feature_name': col,
            'feature_value': value,
            'encoded_value': enc
        })
    
    # Добавим __DEFAULT__ значение на случай NaN
    mte_records.append({
        'feature_name': col,
        'feature_value': '__DEFAULT__',
        'encoded_value': default_tfidf
    })

In [ ]:
# Сводим все MTE-маппинги в табличку (для БД)
mte_df = pd.DataFrame(mte_records)
mte_df.head(10)

,feature_name,feature_value,encoded_value
0,country,Azerbaijan,0.082440
1,country,Belarus,0.183805
2,country,Cyprus,0.121271
3,country,Estonia,0.130912
4,country,Finland,0.133570
5,country,Kazakhstan,0.164002
6,country,Latvia,0.144345
7,country,Russia,0.109255
8,country,Switzerland,0.068321
9,country,Turkey,0.214121


In [ ]:
# Проверяем, что после MTE в трейне остались нужные признаки
df_train_tfidf.head()

,user_id,post_id,target,gender,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,day,month,dayofweek,country_mte,city_mte,age_mte
0,51533,1784,1,1,1,1,0,1,0,0,-0.181055,-0.288288,-0.422105,-0.361728,-0.823934,-1.163084,-0.457821,-0.296544,0.677534,-0.033938,0.752776,0.028988,-3.478632,5.554537,-1.735239,-0.760399,1.942214,0.490945,-2.612931,1.566407,0,0,0,0,1,0,21,10,3,0.171745,0.213132,0.116027
1,51533,1344,0,1,1,1,0,1,0,0,-0.244778,1.171943,-0.487945,-1.144365,-1.489284,-2.875986,0.073866,0.703234,5.743786,2.525433,-2.229066,-2.814638,-13.018264,-6.145329,4.917475,0.076079,-2.851801,-1.665349,0.203243,-0.832864,0,0,0,1,0,0,21,10,3,0.171770,0.215631,0.116619
2,51533,1911,0,1,1,1,0,1,0,0,-0.319494,-0.416234,-0.394980,-0.443572,-0.819001,-1.064639,-0.460328,-0.144719,0.380468,0.300065,-0.233650,-0.392548,-2.261238,-0.010170,-1.342536,0.695462,0.764987,0.170401,-1.089159,1.229232,0,0,0,0,1,0,21,10,3,0.172097,0.216444,0.116285
3,51533,1622,0,1,1,1,0,1,0,0,-0.325991,-0.139774,-0.294887,-0.435077,-0.873326,-0.992215,-0.506083,-0.076823,0.718392,0.247727,0.390431,-0.215585,-2.890603,3.255657,-1.665595,0.161599,0.990986,0.202333,-0.833970,0.751094,0,0,0,0,1,0,21,10,3,0.172097,0.216444,0.116285
4,51533,1211,0,1,1,1,0,1,0,0,-0.315246,0.320371,-0.555416,-0.796566,-1.013993,-1.516432,0.257158,0.613133,3.411011,0.957152,-0.587802,-1.599982,-7.393958,-3.175255,2.470751,0.243670,-1.128147,-0.139595,-0.258421,-0.110980,0,0,0,1,0,0,21,10,3,0.172829,0.218048,0.116532


In [32]:
# Добавим OHE для dayofweek
df_train_tfidf = pd.get_dummies(df_train_tfidf, columns=['dayofweek'],
                          prefix="dayofweek",
                          drop_first=True)
df_test_tfidf = pd.get_dummies(df_test_tfidf, columns=['dayofweek'],
                         prefix="dayofweek",
                         drop_first=True)

In [33]:
# Циклическое кодирование для day
df_train_tfidf['day_sin'] = np.sin(2 * np.pi * df_train_tfidf['day'] / 31)
df_train_tfidf['day_cos'] = np.cos(2 * np.pi * df_train_tfidf['day'] / 31)

df_test_tfidf['day_sin'] = np.sin(2 * np.pi * df_test_tfidf['day'] / 31)
df_test_tfidf['day_cos'] = np.cos(2 * np.pi * df_test_tfidf['day'] / 31)

In [34]:
# Циклическое кодирование для month
df_train_tfidf['month_sin'] = np.sin(2 * np.pi * df_train_tfidf['month'] / 12)
df_train_tfidf['month_cos'] = np.cos(2 * np.pi * df_train_tfidf['month'] / 12)

df_test_tfidf['month_sin'] = np.sin(2 * np.pi * df_test_tfidf['month'] / 12)
df_test_tfidf['month_cos'] = np.cos(2 * np.pi * df_test_tfidf['month'] / 12)

In [ ]:
# Убираем оригинальные признаки day/month
df_train_tfidf = df_train_tfidf.drop(['day', 'month'], axis=1)
df_test_tfidf = df_test_tfidf.drop(['day', 'month'], axis=1)

In [36]:
# Удаление идентификаторов из тренировочного и тестового датасетов
df_train_tfidf = df_train_tfidf.drop(['user_id', 'post_id'], axis=1)
df_test_tfidf = df_test_tfidf.drop(['user_id', 'post_id'], axis=1)

In [ ]:
# Быстрый просмотр финальной матрицы признаков (контрольная ветка)
df_train_tfidf.head()

,target,gender,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,country_mte,city_mte,age_mte,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,day_sin,day_cos,month_sin,month_cos
0,1,1,1,1,0,1,0,0,-0.181055,-0.288288,-0.422105,-0.361728,-0.823934,-1.163084,-0.457821,-0.296544,0.677534,-0.033938,0.752776,0.028988,-3.478632,5.554537,-1.735239,-0.760399,1.942214,0.490945,-2.612931,1.566407,0,0,0,0,1,0,0.171745,0.213132,0.116027,0,0,1,0,0,0,-0.897805,-0.440394,-0.866025,0.5
1,0,1,1,1,0,1,0,0,-0.244778,1.171943,-0.487945,-1.144365,-1.489284,-2.875986,0.073866,0.703234,5.743786,2.525433,-2.229066,-2.814638,-13.018264,-6.145329,4.917475,0.076079,-2.851801,-1.665349,0.203243,-0.832864,0,0,0,1,0,0,0.171770,0.215631,0.116619,0,0,1,0,0,0,-0.897805,-0.440394,-0.866025,0.5
2,0,1,1,1,0,1,0,0,-0.319494,-0.416234,-0.394980,-0.443572,-0.819001,-1.064639,-0.460328,-0.144719,0.380468,0.300065,-0.233650,-0.392548,-2.261238,-0.010170,-1.342536,0.695462,0.764987,0.170401,-1.089159,1.229232,0,0,0,0,1,0,0.172097,0.216444,0.116285,0,0,1,0,0,0,-0.897805,-0.440394,-0.866025,0.5
3,0,1,1,1,0,1,0,0,-0.325991,-0.139774,-0.294887,-0.435077,-0.873326,-0.992215,-0.506083,-0.076823,0.718392,0.247727,0.390431,-0.215585,-2.890603,3.255657,-1.665595,0.161599,0.990986,0.202333,-0.833970,0.751094,0,0,0,0,1,0,0.172097,0.216444,0.116285,0,0,1,0,0,0,-0.897805,-0.440394,-0.866025,0.5
4,0,1,1,1,0,1,0,0,-0.315246,0.320371,-0.555416,-0.796566,-1.013993,-1.516432,0.257158,0.613133,3.411011,0.957152,-0.587802,-1.599982,-7.393958,-3.175255,2.470751,0.243670,-1.128147,-0.139595,-0.258421,-0.110980,0,0,0,1,0,0,0.172829,0.218048,0.116532,0,0,1,0,0,0,-0.897805,-0.440394,-0.866025,0.5


In [ ]:
# Формируем X/y для обучения и валидации (контроль)
X_train = df_train_tfidf.drop('target', axis=1)
y_train = df_train_tfidf['target']
X_test = df_test_tfidf.drop('target', axis=1)
y_test = df_test_tfidf['target']

In [ ]:
# Сохраняем датасеты контрольной ветки
X_train.to_csv('X_train_tfidf.csv', index=False)
y_train.to_csv('y_train_tfidf.csv', index=False)
X_test.to_csv('X_test_tfidf.csv', index=False)
y_test.to_csv('y_test_tfidf.csv', index=False)

## 2 вариант (тест): BERT-эмбеддинги + календарные фичи + MTE

In [ ]:
# Джойним user к логам (аналогично контрольной ветке)
df_bert = pd.merge(
    filtered_sample,
    user,
    on='user_id',
    how='left'
)

In [ ]:
# Джойним постовые фичи BERT
df_bert = pd.merge(
    df_bert,
    post_bert.drop(['text', 'topic'], axis=1),
    on='post_id',
    how='left'
)

In [ ]:
# Быстрый просмотр результата
df_bert.head()

,user_id,post_id,timestamp,target,gender,age,country,city,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4,pca_bert_1,pca_bert_2,pca_bert_3,pca_bert_4,pca_bert_5,pca_bert_6,pca_bert_7,pca_bert_8,pca_bert_9,pca_bert_10,pca_bert_11,pca_bert_12,pca_bert_13,pca_bert_14,pca_bert_15,pca_bert_16,pca_bert_17,pca_bert_18,pca_bert_19,pca_bert_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,51533,1784,2021-10-21 11:16:14,1,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.255103,1.147551,-1.369764,-1.136320,0.130935,0.192699,0.567481,0.390507,0.173209,0.288996,0.933982,-0.636796,0.566059,0.635021,-0.622027,-1.082349,0.073139,-0.067691,-0.202962,-0.023963,0,0,0,0,1,0
1,51533,1344,2021-10-21 11:18:18,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.499371,2.161100,0.368009,1.075422,-0.198104,-0.110495,-0.541164,-0.147317,-0.339600,-0.288608,-0.418770,-0.180329,0.182947,0.113499,-0.108215,0.199472,0.362565,0.173364,-0.045858,0.184346,0,0,0,1,0,0
2,51533,1911,2021-10-21 11:19:03,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.198687,1.010364,-1.251792,-0.201506,0.217381,0.087818,-0.050747,0.271647,-0.364776,0.295530,0.454261,-0.081409,0.607765,0.359567,-0.123839,-0.479638,0.133925,0.385171,0.242253,0.382350,0,0,0,0,1,0
3,51533,1622,2021-10-21 11:20:33,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.149060,1.414328,-1.519578,-0.054354,0.519933,0.123775,0.679006,0.332529,0.578669,-0.150936,-0.536370,0.096971,-0.153550,-0.016774,0.515122,0.435457,0.323496,0.094641,0.013656,-0.246948,0,0,0,0,1,0
4,51533,1211,2021-10-21 11:21:33,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.380358,1.726689,0.495468,0.467069,-0.124328,-0.364501,-0.195681,-0.046997,-0.213835,0.086203,-0.154361,-0.208155,0.170848,-0.153413,0.023794,-0.131188,0.088617,-0.294009,0.065197,0.173764,0,0,0,1,0,0


In [ ]:
# Календарные признаки из timestamp
df_bert['day'] = df_bert['timestamp'].dt.day
df_bert['month'] = df_bert['timestamp'].dt.month
df_bert['dayofweek'] = df_bert['timestamp'].dt.dayofweek
df_bert.head()

,user_id,post_id,timestamp,target,gender,age,country,city,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4,pca_bert_1,pca_bert_2,pca_bert_3,pca_bert_4,pca_bert_5,pca_bert_6,pca_bert_7,pca_bert_8,pca_bert_9,pca_bert_10,pca_bert_11,pca_bert_12,pca_bert_13,pca_bert_14,pca_bert_15,pca_bert_16,pca_bert_17,pca_bert_18,pca_bert_19,pca_bert_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,day,month,dayofweek
0,51533,1784,2021-10-21 11:16:14,1,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.255103,1.147551,-1.369764,-1.136320,0.130935,0.192699,0.567481,0.390507,0.173209,0.288996,0.933982,-0.636796,0.566059,0.635021,-0.622027,-1.082349,0.073139,-0.067691,-0.202962,-0.023963,0,0,0,0,1,0,21,10,3
1,51533,1344,2021-10-21 11:18:18,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.499371,2.161100,0.368009,1.075422,-0.198104,-0.110495,-0.541164,-0.147317,-0.339600,-0.288608,-0.418770,-0.180329,0.182947,0.113499,-0.108215,0.199472,0.362565,0.173364,-0.045858,0.184346,0,0,0,1,0,0,21,10,3
2,51533,1911,2021-10-21 11:19:03,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.198687,1.010364,-1.251792,-0.201506,0.217381,0.087818,-0.050747,0.271647,-0.364776,0.295530,0.454261,-0.081409,0.607765,0.359567,-0.123839,-0.479638,0.133925,0.385171,0.242253,0.382350,0,0,0,0,1,0,21,10,3
3,51533,1622,2021-10-21 11:20:33,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.149060,1.414328,-1.519578,-0.054354,0.519933,0.123775,0.679006,0.332529,0.578669,-0.150936,-0.536370,0.096971,-0.153550,-0.016774,0.515122,0.435457,0.323496,0.094641,0.013656,-0.246948,0,0,0,0,1,0,21,10,3
4,51533,1211,2021-10-21 11:21:33,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.380358,1.726689,0.495468,0.467069,-0.124328,-0.364501,-0.195681,-0.046997,-0.213835,0.086203,-0.154361,-0.208155,0.170848,-0.153413,0.023794,-0.131188,0.088617,-0.294009,0.065197,0.173764,0,0,0,1,0,0,21,10,3


In [ ]:
# Оценка числа дублей с учётом таргета — предварительная диагностика
df_bert.drop(['user_id', 'post_id', 'timestamp'], axis=1).duplicated().sum()

5492

In [ ]:
# Оценка числа дублей без таргета — ищем конфликтные пары (одинаковые фичи, разные target)
df_bert.drop(['user_id', 'post_id', 'timestamp', 'target'], axis=1).duplicated().sum()

7340

In [ ]:
# Удаляем точные дубликаты
df_bert = df_bert.drop_duplicates(subset=df_bert.columns.difference(['user_id', 'post_id']))

In [ ]:
# Находим конфликтные дубли (одинаковые фичи, разные таргеты) и убираем их полностью
duplicated_indices = df_bert.duplicated(subset=df_bert.columns.difference(['user_id', 'post_id', 'timestamp', 'target']), keep=False)
df_bert = df_bert[~duplicated_indices]

In [ ]:
# Перепроверяем дубли с учётом таргета — должно быть 0
df_bert.drop(['user_id', 'post_id', 'timestamp'], axis=1).duplicated().sum()

0

In [ ]:
# Перепроверяем дубли без таргета — также 0
df_bert.drop(['user_id', 'post_id', 'timestamp', 'target'], axis=1).duplicated().sum()

0

In [ ]:
# Диапазон дат в сэмпле — для настройки temporal split
min(df_bert['timestamp']), max(df_bert['timestamp'])

(Timestamp('2021-10-01 06:01:40'), Timestamp('2021-12-29 23:43:15'))

In [ ]:
# Разбивка по времени: train < 2021-12-16, test >= 2021-12-16 (имитируем будущее)
df_train_bert = df_bert[df_bert['timestamp'] < '2021-12-16'].drop('timestamp', axis=1)
df_test_bert = df_bert[df_bert['timestamp'] >= '2021-12-16'].drop('timestamp', axis=1)

In [ ]:
# Проверка структуры train-части до кодирования MTE
df_train_bert.head()

,user_id,post_id,target,gender,age,country,city,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4,pca_bert_1,pca_bert_2,pca_bert_3,pca_bert_4,pca_bert_5,pca_bert_6,pca_bert_7,pca_bert_8,pca_bert_9,pca_bert_10,pca_bert_11,pca_bert_12,pca_bert_13,pca_bert_14,pca_bert_15,pca_bert_16,pca_bert_17,pca_bert_18,pca_bert_19,pca_bert_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,day,month,dayofweek
0,51533,1784,1,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.255103,1.147551,-1.369764,-1.136320,0.130935,0.192699,0.567481,0.390507,0.173209,0.288996,0.933982,-0.636796,0.566059,0.635021,-0.622027,-1.082349,0.073139,-0.067691,-0.202962,-0.023963,0,0,0,0,1,0,21,10,3
1,51533,1344,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.499371,2.161100,0.368009,1.075422,-0.198104,-0.110495,-0.541164,-0.147317,-0.339600,-0.288608,-0.418770,-0.180329,0.182947,0.113499,-0.108215,0.199472,0.362565,0.173364,-0.045858,0.184346,0,0,0,1,0,0,21,10,3
2,51533,1911,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.198687,1.010364,-1.251792,-0.201506,0.217381,0.087818,-0.050747,0.271647,-0.364776,0.295530,0.454261,-0.081409,0.607765,0.359567,-0.123839,-0.479638,0.133925,0.385171,0.242253,0.382350,0,0,0,0,1,0,21,10,3
3,51533,1622,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.149060,1.414328,-1.519578,-0.054354,0.519933,0.123775,0.679006,0.332529,0.578669,-0.150936,-0.536370,0.096971,-0.153550,-0.016774,0.515122,0.435457,0.323496,0.094641,0.013656,-0.246948,0,0,0,0,1,0,21,10,3
4,51533,1211,0,1,25-35,Ukraine,Kherson,1,1,0,1,0,0,-0.380358,1.726689,0.495468,0.467069,-0.124328,-0.364501,-0.195681,-0.046997,-0.213835,0.086203,-0.154361,-0.208155,0.170848,-0.153413,0.023794,-0.131188,0.088617,-0.294009,0.065197,0.173764,0,0,0,1,0,0,21,10,3


In [ ]:
# Функция MTE с KFold (кодируем на трейне, применяем к тесту)
from sklearn.model_selection import KFold

def mean_target_encoding(train, test, col_name, target_name="target", n_folds=5):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    mean_encoded_col = np.zeros(train.shape[0])

    for train_idx, val_idx in kf.split(train):
        train_fold, val_fold = train.iloc[train_idx], train.iloc[val_idx]

        # Вычислим среднеее по таргету для текущих фолдов
        means = train_fold.groupby(col_name)[target_name].mean()

        # Присвоим кодировки для валидационной выборки
        mean_encoded_col[val_idx] = val_fold[col_name].map(means)
    
    # Добавим закодированный столбец в тренировочную выборку
    train[f"{col_name}_mte"] = mean_encoded_col

    # Для теста применим кодировку, полученную на трейне
    global_means =train.groupby(col_name)[target_name].mean()
    test[f"{col_name}_mte"] = test[col_name].map(global_means)

    # Заполним наны средним по всем данным
    overall_mean = train[target_name].mean()
    test[f"{col_name}_mte"].fillna(overall_mean, inplace=True)

    # Дропаем оригинальные колонки
    train.drop([col_name], axis=1, inplace=True)
    test.drop([col_name], axis=1, inplace=True)

    return train, test, global_means.to_dict(), overall_mean

In [ ]:
# Список категориальных полей для MTE + сбор маппингов в отдельный датафрейм
columns_to_encode = ['country', 'city', 'age']
mte_records = []

In [ ]:
# Применяем MTE последовательно и сохраняем маппинги (включая DEFAULT)
for col in columns_to_encode:
    df_train_bert, df_test_bert, mapping_bert, default_bert = mean_target_encoding(df_train_bert, df_test_bert, col)

    # Добавим значения в список записей
    for value, enc in mapping_bert.items():
        mte_records.append({
            'feature_name': col,
            'feature_value': value,
            'encoded_value': enc
        })
    
    # Добавим __DEFAULT__ значение на случай NaN
    mte_records.append({
        'feature_name': col,
        'feature_value': '__DEFAULT__',
        'encoded_value': default_bert
    })

In [ ]:
# Сводим все MTE-маппинги в табличку (для БД)
mte_df = pd.DataFrame(mte_records)
mte_df.head(10)

,feature_name,feature_value,encoded_value
0,country,Azerbaijan,0.082440
1,country,Belarus,0.183809
2,country,Cyprus,0.121271
3,country,Estonia,0.130912
4,country,Finland,0.133570
5,country,Kazakhstan,0.163998
6,country,Latvia,0.144345
7,country,Russia,0.109256
8,country,Switzerland,0.068321
9,country,Turkey,0.214112


In [ ]:
# Проверяем, что после MTE в трейне остались нужные признаки
df_train_bert.head()

,user_id,post_id,target,gender,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4,pca_bert_1,pca_bert_2,pca_bert_3,pca_bert_4,pca_bert_5,pca_bert_6,pca_bert_7,pca_bert_8,pca_bert_9,pca_bert_10,pca_bert_11,pca_bert_12,pca_bert_13,pca_bert_14,pca_bert_15,pca_bert_16,pca_bert_17,pca_bert_18,pca_bert_19,pca_bert_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,day,month,dayofweek,country_mte,city_mte,age_mte
0,51533,1784,1,1,1,1,0,1,0,0,-0.255103,1.147551,-1.369764,-1.136320,0.130935,0.192699,0.567481,0.390507,0.173209,0.288996,0.933982,-0.636796,0.566059,0.635021,-0.622027,-1.082349,0.073139,-0.067691,-0.202962,-0.023963,0,0,0,0,1,0,21,10,3,0.172606,0.211752,0.116387
1,51533,1344,0,1,1,1,0,1,0,0,-0.499371,2.161100,0.368009,1.075422,-0.198104,-0.110495,-0.541164,-0.147317,-0.339600,-0.288608,-0.418770,-0.180329,0.182947,0.113499,-0.108215,0.199472,0.362565,0.173364,-0.045858,0.184346,0,0,0,1,0,0,21,10,3,0.171903,0.220567,0.116304
2,51533,1911,0,1,1,1,0,1,0,0,-0.198687,1.010364,-1.251792,-0.201506,0.217381,0.087818,-0.050747,0.271647,-0.364776,0.295530,0.454261,-0.081409,0.607765,0.359567,-0.123839,-0.479638,0.133925,0.385171,0.242253,0.382350,0,0,0,0,1,0,21,10,3,0.172218,0.217622,0.116392
3,51533,1622,0,1,1,1,0,1,0,0,-0.149060,1.414328,-1.519578,-0.054354,0.519933,0.123775,0.679006,0.332529,0.578669,-0.150936,-0.536370,0.096971,-0.153550,-0.016774,0.515122,0.435457,0.323496,0.094641,0.013656,-0.246948,0,0,0,0,1,0,21,10,3,0.172218,0.217622,0.116392
4,51533,1211,0,1,1,1,0,1,0,0,-0.380358,1.726689,0.495468,0.467069,-0.124328,-0.364501,-0.195681,-0.046997,-0.213835,0.086203,-0.154361,-0.208155,0.170848,-0.153413,0.023794,-0.131188,0.088617,-0.294009,0.065197,0.173764,0,0,0,1,0,0,21,10,3,0.171691,0.219323,0.116365


In [58]:
# Добавим OHE для dayofweek
df_train_bert = pd.get_dummies(df_train_bert, columns=['dayofweek'],
                          prefix="dayofweek",
                          drop_first=True)
df_test_bert = pd.get_dummies(df_test_bert, columns=['dayofweek'],
                         prefix="dayofweek",
                         drop_first=True)

In [59]:
# Циклическое кодирование для day
df_train_bert['day_sin'] = np.sin(2 * np.pi * df_train_bert['day'] / 31)
df_train_bert['day_cos'] = np.cos(2 * np.pi * df_train_bert['day'] / 31)

df_test_bert['day_sin'] = np.sin(2 * np.pi * df_test_bert['day'] / 31)
df_test_bert['day_cos'] = np.cos(2 * np.pi * df_test_bert['day'] / 31)

In [60]:
# Циклическое кодирование для month
df_train_bert['month_sin'] = np.sin(2 * np.pi * df_train_bert['month'] / 12)
df_train_bert['month_cos'] = np.cos(2 * np.pi * df_train_bert['month'] / 12)

df_test_bert['month_sin'] = np.sin(2 * np.pi * df_test_bert['month'] / 12)
df_test_bert['month_cos'] = np.cos(2 * np.pi * df_test_bert['month'] / 12)

In [ ]:
# Убираем оригинальные признаки day/month
df_train_bert = df_train_bert.drop(['day', 'month'], axis=1)
df_test_bert = df_test_bert.drop(['day', 'month'], axis=1)

In [62]:
# Удаление идентификаторов из тренировочного и тестового датасетов
df_train_bert = df_train_bert.drop(['user_id', 'post_id'], axis=1)
df_test_bert = df_test_bert.drop(['user_id', 'post_id'], axis=1)

In [ ]:
# Быстрый просмотр финальной матрицы признаков (тестовая ветка)
df_train_bert.head()

,target,gender,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4,pca_bert_1,pca_bert_2,pca_bert_3,pca_bert_4,pca_bert_5,pca_bert_6,pca_bert_7,pca_bert_8,pca_bert_9,pca_bert_10,pca_bert_11,pca_bert_12,pca_bert_13,pca_bert_14,pca_bert_15,pca_bert_16,pca_bert_17,pca_bert_18,pca_bert_19,pca_bert_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech,country_mte,city_mte,age_mte,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,day_sin,day_cos,month_sin,month_cos
0,1,1,1,1,0,1,0,0,-0.255103,1.147551,-1.369764,-1.136320,0.130935,0.192699,0.567481,0.390507,0.173209,0.288996,0.933982,-0.636796,0.566059,0.635021,-0.622027,-1.082349,0.073139,-0.067691,-0.202962,-0.023963,0,0,0,0,1,0,0.172606,0.211752,0.116387,0,0,1,0,0,0,-0.897805,-0.440394,-0.866025,0.5
1,0,1,1,1,0,1,0,0,-0.499371,2.161100,0.368009,1.075422,-0.198104,-0.110495,-0.541164,-0.147317,-0.339600,-0.288608,-0.418770,-0.180329,0.182947,0.113499,-0.108215,0.199472,0.362565,0.173364,-0.045858,0.184346,0,0,0,1,0,0,0.171903,0.220567,0.116304,0,0,1,0,0,0,-0.897805,-0.440394,-0.866025,0.5
2,0,1,1,1,0,1,0,0,-0.198687,1.010364,-1.251792,-0.201506,0.217381,0.087818,-0.050747,0.271647,-0.364776,0.295530,0.454261,-0.081409,0.607765,0.359567,-0.123839,-0.479638,0.133925,0.385171,0.242253,0.382350,0,0,0,0,1,0,0.172218,0.217622,0.116392,0,0,1,0,0,0,-0.897805,-0.440394,-0.866025,0.5
3,0,1,1,1,0,1,0,0,-0.149060,1.414328,-1.519578,-0.054354,0.519933,0.123775,0.679006,0.332529,0.578669,-0.150936,-0.536370,0.096971,-0.153550,-0.016774,0.515122,0.435457,0.323496,0.094641,0.013656,-0.246948,0,0,0,0,1,0,0.172218,0.217622,0.116392,0,0,1,0,0,0,-0.897805,-0.440394,-0.866025,0.5
4,0,1,1,1,0,1,0,0,-0.380358,1.726689,0.495468,0.467069,-0.124328,-0.364501,-0.195681,-0.046997,-0.213835,0.086203,-0.154361,-0.208155,0.170848,-0.153413,0.023794,-0.131188,0.088617,-0.294009,0.065197,0.173764,0,0,0,1,0,0,0.171691,0.219323,0.116365,0,0,1,0,0,0,-0.897805,-0.440394,-0.866025,0.5


In [ ]:
# Формируем X/y для обучения и валидации (тест)
X_train = df_train_bert.drop('target', axis=1)
y_train = df_train_bert['target']
X_test = df_test_bert.drop('target', axis=1)
y_test = df_test_bert['target']

In [ ]:
# Сохраняем датасеты тестовой ветки
X_train.to_csv('X_train_bert.csv', index=False)
y_train.to_csv('y_train_bert.csv', index=False)
X_test.to_csv('X_test_bert.csv', index=False)
y_test.to_csv('y_test_bert.csv', index=False)

Подготовка таблиц для работы сервиса по предсказаниям

In [ ]:
# Подключение к БД и запись таблиц (названия соответствуют моему логину на портале)
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

user.to_sql(
    'tikhonovrs96_features_user_lesson_22',
    con=engine,
    if_exists='replace',
    index=False
)

post_tfidf.to_sql(
    'tikhonovrs96_features_post_tfidf_lesson_22',
    con=engine,
    if_exists='replace',
    index=False
)

post_bert.to_sql(
    'tikhonovrs96_features_post_bert_lesson_22',
    con=engine,
    if_exists='replace',
    index=False
)

mte_df.to_sql(
    'tikhonovrs96_features_mte_lesson_22',
    con=engine,
    if_exists='replace',
    index=False
)

908

In [ ]:
# Быстрая проверка чтения user-фичей из БД
data_user = pd.read_sql('SELECT * FROM tikhonovrs96_features_user_lesson_22', con=engine) # считываем таблицу
data_user

,user_id,gender,age,country,city,is_android,is_ads,exp_group_1,exp_group_2,exp_group_3,exp_group_4
0,200,1,25-35,Russia,Degtyarsk,1,1,0,0,1,0
1,201,0,35-45,Russia,Abakan,1,1,0,0,0,0
2,202,1,до 18,Russia,Smolensk,1,1,0,0,0,1
3,203,0,18-25,Russia,Moscow,0,1,1,0,0,0
4,204,0,35-45,Russia,Anzhero-Sudzhensk,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
163200,168548,0,35-45,Russia,Kaliningrad,1,0,0,0,0,1
163201,168549,0,18-25,Russia,Tula,1,0,0,1,0,0
163202,168550,1,35-45,Russia,Yekaterinburg,1,0,0,0,0,1
163203,168551,0,35-45,Russia,Moscow,0,0,0,0,1,0


In [ ]:
# Проверка чтения постовых фичей TF-IDF из БД
data_post_tfidf = pd.read_sql('SELECT * FROM tikhonovrs96_features_post_tfidf_lesson_22', con=engine)
data_post_bert = pd.read_sql('SELECT * FROM tikhonovrs96_features_post_bert_lesson_22', con=engine)
data_post_tfidf

,post_id,text,topic,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,UK economy facing major risks\n\nThe UK manufa...,business,-0.375438,-0.211312,-0.967604,-1.289273,-1.382435,-2.553740,-0.198691,-0.188618,3.623218,1.806545,-1.646793,-2.425448,-11.143379,-4.729736,1.600485,0.417081,-1.371517,-1.062643,-1.383139,0.371786,0,0,0,0,0,0
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,-0.350043,0.307126,-0.772762,-1.416407,-1.268760,-2.934686,-0.098141,0.493644,4.668232,2.464402,-2.500853,-2.956410,-13.174602,-4.469178,4.403922,-0.354127,-2.332844,-1.885773,-1.192080,-0.109992,0,0,0,0,0,0
2,3,Asian quake hits European shares\n\nShares in ...,business,-0.453684,0.582027,-2.044489,-1.944706,-2.262692,-4.412119,-0.082363,-0.336395,6.080126,2.691513,-2.715423,-4.328486,-21.821127,-10.250555,6.376745,-2.394627,-2.662655,-2.568979,-4.505346,0.774517,0,0,0,0,0,0
3,4,India power shares jump on debut\n\nShares in ...,business,-0.434171,-0.329674,-0.944819,-1.250985,-1.603222,-2.199247,-0.332843,-0.677520,1.180545,1.072667,-1.632867,-1.907533,-7.491799,-5.396203,-1.285849,1.270082,-0.944739,-0.786775,-0.623056,0.874489,0,0,0,0,0,0
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,-0.413180,-0.324597,-0.761652,-0.715293,-1.011243,-0.915354,-0.008083,-0.592711,0.151614,0.561311,-1.166500,-1.093569,-3.996585,-3.745349,-1.590957,1.237534,-0.597091,-0.467014,-0.170502,0.143495,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7314,Although the likeliness of someone focusing on...,movie,0.252313,0.171291,0.815593,0.699774,1.529001,2.893943,0.373136,0.917887,-1.093681,-0.890958,0.956608,1.349216,6.734710,1.380840,3.367196,-2.438107,-0.409987,1.064126,0.594974,-1.203363,0,0,1,0,0,0
7019,7315,"OK, I would not normally watch a Farrelly brot...",movie,-0.042354,-0.040702,0.087203,0.216829,0.591192,1.262353,-0.144690,-0.147608,-1.798745,-0.794568,0.692884,1.090368,5.583468,0.828375,-0.557229,-0.607419,0.317327,0.788880,0.058961,0.064494,0,0,1,0,0,0
7020,7317,I cant believe this film was allowed to be mad...,movie,-0.059922,-0.359106,-0.013607,0.065544,0.319317,0.610259,-0.162239,0.023932,-1.544246,-0.799549,0.566564,0.660018,4.434857,0.482239,-1.202285,-0.195191,0.174078,0.808574,0.340312,-0.291335,0,0,1,0,0,0
7021,7318,The version I saw of this film was the Blockbu...,movie,0.099954,-0.309059,0.029884,0.245311,0.507322,0.677401,0.065500,0.063338,-0.958680,-0.721816,0.425905,0.724864,3.741371,0.989264,0.295327,-0.771321,0.472547,0.446173,0.391909,-0.144616,0,0,1,0,0,0


In [ ]:
# Проверка чтения постовых фичей BERT из БД
data_post_bert

,post_id,text,topic,pca_bert_1,pca_bert_2,pca_bert_3,pca_bert_4,pca_bert_5,pca_bert_6,pca_bert_7,pca_bert_8,pca_bert_9,pca_bert_10,pca_bert_11,pca_bert_12,pca_bert_13,pca_bert_14,pca_bert_15,pca_bert_16,pca_bert_17,pca_bert_18,pca_bert_19,pca_bert_20,topic_covid,topic_entertainment,topic_movie,topic_politics,topic_sport,topic_tech
0,1,UK economy facing major risks\n\nThe UK manufa...,business,-0.789074,1.578995,1.421159,0.292831,0.196380,0.482332,-0.172336,0.252792,0.128668,-0.029784,-0.456210,-0.536405,0.113529,0.143566,0.053079,0.060490,-0.177239,-0.033712,-0.580020,-0.006120,0,0,0,0,0,0
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,-0.792764,1.521056,0.897746,-0.228620,0.109591,-0.125362,-0.163190,-0.005947,-0.349708,-0.423531,0.408382,0.690819,0.125367,0.447225,-0.153495,0.281642,0.264169,0.326188,-0.133573,-0.337692,0,0,0,0,0,0
2,3,Asian quake hits European shares\n\nShares in ...,business,-0.801292,1.223608,0.669452,-1.323420,-0.081454,-0.550803,0.049824,0.241455,-0.509045,-0.007866,-0.678881,-0.182175,0.202675,-0.066136,0.210822,0.072629,-0.023954,-0.078617,-0.773398,0.171038,0,0,0,0,0,0
3,4,India power shares jump on debut\n\nShares in ...,business,-0.867122,0.971222,1.577188,-0.833192,0.841209,0.292207,0.742447,-0.886750,-0.494585,0.055655,-0.567772,0.230937,-0.453456,0.290870,0.044735,-0.518971,-0.258113,0.583832,-0.053154,-0.287652,0,0,0,0,0,0
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,-0.411659,0.809849,0.682362,-0.790953,-0.186110,0.596705,0.169838,0.758098,-0.193135,0.064067,0.288202,-0.007051,-0.300151,-0.287257,0.124016,-0.170200,-0.403123,-0.233267,0.087435,0.026991,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7314,Although the likeliness of someone focusing on...,movie,0.978513,-0.504691,0.278772,0.149315,-0.452346,-1.069672,0.008570,0.072078,0.418195,0.129565,0.113346,0.081768,-0.051115,-0.022833,0.011907,-0.088118,0.043473,0.272058,-0.305248,-0.247324,0,0,1,0,0,0
7019,7315,"OK, I would not normally watch a Farrelly brot...",movie,0.983509,-0.483879,-0.001963,-0.066758,0.059189,-0.342647,0.026161,0.575154,-0.049793,0.668213,0.225396,0.143204,-0.063735,0.365156,-0.025876,0.116609,-0.394875,0.124753,-0.223166,-0.039118,0,0,1,0,0,0
7020,7317,I cant believe this film was allowed to be mad...,movie,0.469400,-0.874857,-0.482077,0.138474,-0.257596,-0.545992,-0.031882,0.287168,-0.504227,0.519209,-0.220130,-0.044093,-0.063617,-0.193567,-0.108290,-0.130713,0.403388,0.297430,-0.179709,0.180410,0,0,1,0,0,0
7021,7318,The version I saw of this film was the Blockbu...,movie,1.579792,-0.448613,-0.126692,-0.100854,0.284518,0.370505,-0.190609,0.049485,-0.189625,0.021933,-0.210885,-0.288475,-0.277857,0.454060,-0.071188,0.129983,-0.086229,-0.239920,0.228375,0.118058,0,0,1,0,0,0


In [ ]:
# Проверка чтения MTE-маппингов (используются в приложении при онлайн кодировании)
data_mte = pd.read_sql('SELECT * FROM tikhonovrs96_features_mte_lesson_22', con=engine) # считываем таблицу
data_mte

,feature_name,feature_value,encoded_value
0,country,Azerbaijan,0.082440
1,country,Belarus,0.183809
2,country,Cyprus,0.121271
3,country,Estonia,0.130912
4,country,Finland,0.133570
...,...,...,...
1903,age,55-65,0.112383
1904,age,65-75,0.108359
1905,age,75 и выше,0.117363
1906,age,до 18,0.116149


## Итоги и артефакты

**Сформировано две ветки признаков:**
- Контрольная: **TF-IDF -> StandardScaler -> PCA(20) + календарные/OHE + MTE**
- Тестовая: **BERT-эмбеддинги (PCA 20) + календарные/OHE + MTE**

**Сохранено в CSV:**
- `X_train_tfidf.csv`, `y_train_tfidf.csv`, `X_test_tfidf.csv`, `y_test_tfidf.csv`
- `X_train_bert.csv`,  `y_train_bert.csv`,  `X_test_bert.csv`,  `y_test_bert.csv`

**Записано в БД (Postgres):**
- `tikhonovrs96_features_user_lesson_22`
- `tikhonovrs96_features_post_tfidf_lesson_22`
- `tikhonovrs96_features_post_bert_lesson_22`
- `tikhonovrs96_features_mte_lesson_22`

**Важно для сервиса:**
- Признаковое пространство на предикте должно **строго совпадать** с обучением (имена/порядок/тип).  
- MTE-маппинги брать из сохранённой таблицы `*_mte_*` (при отсутствии значения — использовать `__DEFAULT__`).

**Следующий шаг:** обучение моделей на сохранённых `X/y` и интеграция в сервис рекомендаций.
